In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import matplotlib.pyplot as plt


In [1]:
def setread_csvfile(yr,bystate):
        # Import gun violation file as DataFrame
        cities_pd = pd.read_csv("../../../../resources/data/inc_dt_reg_lat_lon.csv")
        cities_pd.head()
        #Convert the data type of Date of Incident
        cities_pd['date'] = pd.to_datetime(cities_pd['date'], errors='coerce')
        cities_pd['Year'] = cities_pd['date'].dt.year
        cities_pd['Month'] = cities_pd['date'].dt.month
        cities_pd['Day'] = cities_pd['date'].dt.day
        # group by State and Year and aggregate sum 
        cities_pd=cities_pd[cities_pd.Year == int(yr)]
        if bystate==True:
            cities_pd= pd.DataFrame({'KilledCount': cities_pd.groupby(['Year','state'])['n_killed'].sum(),
                                   'InjuredCount': cities_pd.groupby(['Year','state'])['n_injured'].sum()})
        else:
            cities_pd= pd.DataFrame({'KilledCount': cities_pd.groupby(['Year','state','city_or_county'])['n_killed'].sum(),
                                   'InjuredCount': cities_pd.groupby(['Year','state','city_or_county'])['n_injured'].sum()})
       
    #remove the state with kiling and injured = 0
        cities_pd=cities_pd[cities_pd.KilledCount != 0] 
        cities_pd=cities_pd[cities_pd.InjuredCount != 0]
        #save it to CSV
        cities_pd.to_csv("inc_dt_reg_lat_lon_new.csv")
        cities_pd = pd.read_csv("inc_dt_reg_lat_lon_new.csv")
        return(cities_pd)

In [ ]:
# for USA map we need this list to conver STATE to 2 letter 
def buildmapbystatevio(cities_pd,yr):
        us_state_abbrev = {
            'Alabama': 'AL',
            'Alaska': 'AK',
            'Arizona': 'AZ',
            'Arkansas': 'AR',
            'California': 'CA',
            'Colorado': 'CO',
            'Connecticut': 'CT',
            'Delaware': 'DE',
            'Florida': 'FL',
            'Georgia': 'GA',
            'Hawaii': 'HI',
            'Idaho': 'ID',
            'Illinois': 'IL',
            'Indiana': 'IN',
            'Iowa': 'IA',
            'Kansas': 'KS',
            'Kentucky': 'KY',
            'Louisiana': 'LA',
            'Maine': 'ME',
            'Maryland': 'MD',
            'Massachusetts': 'MA',
            'Michigan': 'MI',
            'Minnesota': 'MN',
            'Mississippi': 'MS',
            'Missouri': 'MO',
            'Montana': 'MT',
            'Nebraska': 'NE',
            'Nevada': 'NV',
            'New Hampshire': 'NH',
            'New Jersey': 'NJ',
            'New Mexico': 'NM',
            'New York': 'NY',
            'North Carolina': 'NC',
            'North Dakota': 'ND',
            'Ohio': 'OH',
            'Oklahoma': 'OK',
            'Oregon': 'OR',
            'Pennsylvania': 'PA',
            'Rhode Island': 'RI',
            'South Carolina': 'SC',
            'South Dakota': 'SD',
            'Tennessee': 'TN',
            'Texas': 'TX',
            'Utah': 'UT',
            'Vermont': 'VT',
            'Virginia': 'VA',
            'Washington': 'WA',
            'West Virginia': 'WV',
            'Wisconsin': 'WI',
            'Wyoming': 'WY',
        }
  
        cities_pd['code'] = cities_pd['state'].map(us_state_abbrev)

        for col in cities_pd.columns:
            cities_pd[col] = cities_pd[col].astype(str)
        #for filling state color
        #scl = [[0.0, 'rgb(255,255,255)']]

        cities_pd['text'] = cities_pd['state'] + '<br>' +\
            'TotalKilled '+cities_pd['KilledCount']+ '<br>' + 'TotalInjured'+cities_pd['InjuredCount']

        data = [ dict(
                type='choropleth',
                autocolorscale = True,
                locations = cities_pd["code"],
                z = cities_pd['KilledCount'].astype(float),
                locationmode = 'USA-states',
                text = cities_pd['text'],
                marker = dict(
                    line = dict (
                        color = 'rgb(255,255,255)',
                        width = 2
                    ) ),
                colorbar = dict(
                    title = "Killed in Numbers")
                ) ]
        layout = dict(
                title=f"Gun Violations by  State<br>(Hover for breakdown) {yr}",
                geo = dict(
                    scope='usa',
                    projection=dict( type='albers usa' ),
                    showlakes = True,
                    lakecolor = 'rgb(255, 255, 255)'),
                     )

        fig = dict( data=data, layout=layout )
        return fig
        


In [ ]:
# Google API Key
def gun_vio_locations(cities_pd,gkey):
        cities_pd["Lat"] = ""
        cities_pd["Lng"] = ""
        # create a params dict that will be updated with new city each iteration
        params = {"key": gkey}
        # Loop through the cities_pd and run a lat/long search for each city
        for index, row in cities_pd.iterrows():
           base_url = "https://maps.googleapis.com/maps/api/geocode/json"
           city=row['city_or_county']
           state=row['state']
           params['address'] = f"{city},{state}"

           # make request
           cities_lat_lng = requests.get(base_url, params=params)

           # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
           # print(cities_lat_lng.url)

           # convert to json
           cities_lat_lng = cities_lat_lng.json()
           cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
           cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

        # Visualize to confirm lat lng appear

        Killed_Rate = cities_pd["KilledCount"].tolist()

        # Create a map using state centroid coordinates to set markers
        #marker_locations = cities_pd[['Lat', 'Lng']]
        marker_locations = cities_pd[['Lat', 'Lng']]
        gun_vio_layer = gmaps.symbol_layer(
            marker_locations, fill_color='red', stroke_color='red', scale=2
        )
        return gun_vio_layer
      

        #fig = gmaps.figure()
        #markers = gmaps.marker_layer(marker_locations,
         #  info_box_content=[f"Killed Rate: {rate}" for rate in Killed_Rate])
        #fig.add_layer(markers)
        #fig